<a href="https://colab.research.google.com/github/kameda-yoshinari/DataAlgo-UT/blob/main/DataAlgo_UT(016)_FPTAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 8.2. FPTAS

本節では，近似精度と計算量との関係についてさらに学習する．




**いつもの約束**  
１つのコードセルだけの実行は Ctrl + Enter．  
エディタで「インデント幅（スペース）は4で表示」「行番号を表示」「インデントガイドを表示」．  
内部では日本語はUTF-8で表現されている．


# 準備

インスタンスに接続し起動する．  
下記の手順でGoogle Driveをマウントする．  
マウント先に移動し，作業フォルダとする．  
これによって，インスタンスがリセットされてもGoogle Drive内にファイルが保存されるようにする．

In [ ]:
!echo "Google Driveをマウントします"
from google.colab import drive 
drive.mount('/content/drive')

In [ ]:
!echo "今回の作業用フォルダを作成しそこに移動します"
%cd /content/drive/My\ Drive/
%mkdir -p 202004_DataAlgo/DA2021_016
%cd       202004_DataAlgo/DA2021_016
!ls
!echo "日本時間表示"
!rm /etc/localtime
!ln -s /usr/share/zoneinfo/Japan /etc/localtime
!date

# FPTAS

近似解を多項式時間で得る近似アルゴリズムはいつも発明発見できるわけではない．
実際，多くの研究努力がこの近似アルゴリズムの発明発見に費やされている．  

今，最小値問題において，α近似アルゴリズムが見つかったとする．(α>=1)

8.1.節の例ではα=2と固定値であったが，これでは使い勝手がよくない．

そこで，α=1+ε (ε>0) とし，任意のεについて解を求めることができる近似アルゴリズムを，特に Polynomial Time Approximation Scheme (PTAS) と呼ぶ．

これは大変にありがたいアルゴリズムである．

もちろんそんなにうまい話があるはずがなく，多くの場合，ε→0にしていくときに，その小ささ（多くの場合は 1/ε）に対して指数的に計算時間が必要となってしまうことが多い．

ところが，ごく稀にこの 1/ε に対して計算時間の増加が1/εの多項式時間でしか増加しないアルゴリズムが発明発見されることがある．この近似アルゴリズムのことを，特に敬意を込めて，Fully Polynomial Time Approximation Scheme (FPTAS) と呼ぶ．








# ナップサック問題におけるFPTASアルゴリズム

6.3.節で取り上げたn個に対する0-1ナップサック問題について再び取り上げる．

ナップサック問題は最大化問題なので，任意のε(ε>0)について，最適解に対して近似解が 1-ε の近似比率まで近づけることを考える．

ここで，物体 i の価値を Vi, 重さを Wi とする．また，n個の中で(重さに関係なく)最大価値の物品の価値を Vmax とする．


1．K = ε Vmax / n というKを導入する．
2. V'i = floor(Vi / K) とする．ここでfloor関数は小数点以下を切り捨てる関数である．
3. 各物体iが (V'i, Wi) }で表される問題を修正ナップサック問題と呼ぶ．修正ナップサック問題での修正最適解 {X'} を求める．(X'iは 0 か 1)
4. 「修正最適解 {X'i}による価値合計」とVmaxとを比較し，評価値の高いほうを近似解{Xa}とする．

**計算時間**

問題になるのは手順3である．

実は，0-1ナップサック問題は，物体数 n だけではなく，ナップサックの重量制限 w も考慮に入れると，計算時間は O(n w) とすることができる．このことは実際に後でプログラムを見て確認してみよう．

**精度**

解候補を{X}と表す(各要素が0又は1であるn次元ベクトル）．    
V({X})は解{X}に対する元問題での価値合計を示す．  
V'({X})は解{X}に対する修正問題での価値合計を示す．  
{Xopt} は元問題での最適解．  
{X'} は修正問題での最適解（修正最適解)．  
{Xa} は近似解．

- n K = ε Vmax ... 定義より
- V'i = floor(Vi / K) ... 定義より
- 0 < Vi - K V'i < K ... floor関数を展開
- 0 < V({X}) - K V'({X}) < n K ... 上式は n 個まとめても成立
- V'({Xopt}) < V'({X'}) ... 修正問題V'({X})についての最適解は{Xopt}ではなくあくまで{X'}であるため
- Vmax <= V({Xa}) ... 上記で手順4.としているため当然

これらを組み合わせていくことで，以下が導出できる．

- V({Xa}) >= (1 / (1+ε)) V({Xopt}) > (1-ε) V({Xopt})




# ナップサック問題を解くCプログラム

**アルゴリズム**

価値が正整数であることを仮定し，仮想的に「ぴったりM万円」のナップサックを多数そろえているとイメージすると分かりやすいだろう．もしぴったしM万円のナップサックを作る組み合わせが幾つかあるのなら，軽い方がよい．

物品番号0から(L81-82)、順に考慮すべき対象を徐々に(L85のループ)増やしていく．

**計算時間**

計算時間についてこのプログラムを観察すると，O(N * v_range) になっている．  
プログラムに書かれているように，v_range = N * VDasgNax なので，結局 O(N^2 * VDashmax) と言える．

このプログラムは修正問題を扱っているので，プログラム内のVDasgMaxは，元問題の変数で表すと次のようになる．

- VDashMax = floor(Vmax / K)

これはすなわち

- VDashMax = floor(n / ε)

であるから，結局，

- O(n^2 * VDashmax) = O(n^2 * floor(n / ε) = O(n^3 / ε)

つまり n と 1/ε の両方について多項式時間オーダーである． 

**備考**

本プログラムは，動的計画法と呼ばれるアルゴリズム構造をしている．  
Floydのアルゴリズムとの類似性を見てみること．



In [ ]:
%%writefile Knapsack-DP-value.c
// Pseudo-polynomial Time Algorithm for Knapsack
//   for FPTAS Knapsack solution
//   kameda[at]ccs.tsukuba.ac.jp, 2021.
#include <stdio.h>
#include <stdlib.h> // calloc()

// Example-A: max value 280 = (15, 100, 90, 60, 0, 15, 0 ,0) at weight 111
// Example-B: max value  24 = () at weight 17

#define EXAMPLE_A

#ifdef EXAMPLE_A
#define WEIGHT_LIMIT 112
#define N 8 // Number of object classes, Not number of objects
struct {
	int v; // value
	int w; // weight
} obj[N] = {
	{ 15,  6},
	{100, 20},
	{ 90, 25},
	{ 60, 30},
	{ 40, 40},
	{ 15, 30},
	{ 10, 60},
	{  3, 10}
};
#endif

#ifdef EXAMPLE_B
#define WEIGHT_LIMIT 17
#define N 5 // Number of object classes, Not number of objects
struct {
	int v; // value
	int w; // weight
} obj[N] = {
	{  4,  3},
	{  5,  4},
	{ 10,  7},
	{ 11,  8},
	{ 13,  9}
};
#endif

// Pseudo-polynomial Time Algorithm for Knapsack Problem
int knapsack_byvalue(void) {
	int	VDashMax = 0; // largest value among objects
	int v_range = 0; // value range, it should be the sum of all weights (less than largetvalue * N)
	int *ns[N]; // S: ns[i][value], i = 0 - N-1, value = 0 - N * VDashMax : object-ID to pick
	int *nw[N]; // A: nw[i][value], i = 0 - N-1, value = 0 - N * VDashMax : weight of the knapsack, -1:infinite
	int max_v = 0;
	int max_w = 0;
	int i;
	int p;
	int ncalc = 0;

	// Find the largest value
	for (i = 0; i < N; i++)
		if (VDashMax < obj[i].v) VDashMax = obj[i].v;
	v_range = N * VDashMax;

	// Online memory allocation
	for (i = 0; i < N; i++) {
		if ((ns[i] = (int *)calloc(v_range, sizeof(*ns[i]))) == NULL || (nw[i] = (int *)calloc(v_range, sizeof(*ns[i]))) == NULL) {
			printf("Failed to malloc %d integers at %d.\n", v_range, i);
			return -1;
		}
	}

	// Initialization
	for (i = 0; i < N; i++) {
		for (p = 0; p <= N * VDashMax; p++) {
			ns[i][p] = -1;
			nw[i][p] = -1;
		}
	}
	ns[0][0] = -1; // nothing to pick
	nw[0][0] = 0;  // empty knapsack

	ns[0][obj[0].v] = 0; // Object-ID:0
	nw[0][obj[0].v] = obj[0].w; // Initial weight of object-ID:0

	// Core loop
	for (i = 1; i < N; i++) {
		// Do nothing before obj[i].v
		for (p = 0; p < obj[i].v; p++) {
			nw[i][p] = nw[i-1][p];
			ncalc++;
		}
		// Examine to make the knapsack of value=p lighter
		for (p = obj[i].v; p <= v_range; p++) {
			if ((nw[i-1][p - obj[i].v] >= 0) &&
				((nw[i-1][p] >= 0 && nw[i-1][p - obj[i].v] + obj[i].w < nw[i-1][p]) || (nw[i-1][p] < 0))) {
				nw[i][p] = nw[i-1][p - obj[i].v] + obj[i].w;
				ns[i][p] = i; // Pick Object-i here (yes/no)
			} else {
				nw[i][p] = nw[i-1][p];
			}
			ncalc++;
		}

		if (0) { // Check the array if you wish by changing 0 to 1
			int k;
			printf("ObjectID = %d\n", i);
			for (k = 0; k <= v_range; k++) printf("%3d ", k); printf("\n");
			for (k = 0; k <= v_range; k++) 	printf("%3d ", nw[i][k]); printf("\n");
			for (k = 0; k <= v_range; k++) 	printf("%3d ", ns[i][k]); printf("\n");
		}
	}

	// Find the answer
	for (p = 0; p <= v_range; p++) {
		if (max_v < p && nw[N-1][p] <= WEIGHT_LIMIT && nw[N-1][p] >= 0) {
			max_v = p;
			max_w = nw[N-1][p];
		}
	}
	printf("Answer: max_v = %d, max_w = %d\n", max_v, max_w);
	for (i = N - 1, p = max_v; i >= 0; i--) {
		printf("Object %d (total value = %3d): ", i, p);
		if (ns[i][p] >= 0) {
			printf("pick\n");
			p = p - obj[i].v;
		} else {
			printf("----\n");
			p = p; // stand-by at the same value
		}
	}
	if (0) { // To verify ns[N][] and nw[N][]
		int k,ii;
		for (k = 0; k <= v_range; k++) {
			int cc = 0;
			for (ii = 0; ii < N; ii++) {
				if (ns[ii][k] >= 0)
					cc++;
			}
			printf ("%3d ", cc);
		}
	}
	printf("Calculation = %d (Est. %d = %d * %d * %d)\n", ncalc, v_range * N, N, N, VDashMax);
	return 0;
}

// Main function
int main(int argc, char *argv[]) {
	knapsack_byvalue();
	return 0;
}


コンパイル．デバック（学習）用にとコンパクトに書いたL105-17で親切にもWarningが出る（が意図して行っているので無視）．  
(逆に言えばこのレベルの気遣いまで今のコンパイラはしてくれている）

In [ ]:
!gcc -Wall -o Knapsack-DP-value Knapsack-DP-value.c

実行してみよう．EXAMPLE_Aだけでなく，EXAMPLE_Bでも試してみること．

In [ ]:
!./Knapsack-DP-value

時間計算量が頂点数に対して多項式時間であることを確認しておくこと．

余談であるが，巡回セールスマン問題に対してPTASアルゴリズムがあることが発見されたのはつい最近の1998年である．（私がこの授業の受け持つようになったのは2003年）この授業は，それぐらい最近の研究成果にまで近付いているのである．

# 節末課題

1. ナップサック問題におけるFPTASアルゴリズムの精度保証  
ナップサック問題におけるFPTASアルゴリズムの精度の考察について，V({Xa}) > (1-ε) V({Xopt})を実際に導出せよ  


2. 動的計画法アルゴリズムによる類似性
今回の Knapsack-DP-value.c が，以前に授業で行った shortest-floyd_J.c と類似している点について，この二つが同じ動的計画法アルゴリズムであるとの観点から述べよ．









# 出典

筑波大学工学システム学類  
データ構造とアルゴリズム  
担当：亀田能成  
2021/06/23 初版